In [4]:
from azureml.core import Workspace, Experiment
from azureml.pipeline.steps import PythonScriptStep

ws = Workspace.get(name="quick-starts-ws-124826")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()


Workspace name: quick-starts-ws-124826
Azure region: southcentralus
Subscription id: 82648f26-b738-43a4-9ebb-f954c9f1ff3a
Resource group: aml-quickstarts-124826


In [7]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster

# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.
vm_size = "Standard_D2_V2"
compute_name="FirstVM"
# create the cluster
compute_min_nodes=0
compute_max_nodes=4
 



### YOUR CODE HERE ###
if compute_name in ws.compute_targets:
    compute_target = ws.compute_targets[compute_name]
    if compute_target and type(compute_target) is AmlCompute:
        print("found compute target: " + compute_name)
else:
    print("creating new compute target...")
    provisioning_config = AmlCompute.provisioning_configuration(vm_size = vm_size,
                                                                min_nodes = compute_min_nodes, 
                                                                max_nodes = compute_max_nodes)
    # if no min node count is provided it will use the scale settings for the cluster
    compute_target.wait_for_completion(show_output=True, timeout_in_minutes=20)
    # For a more detailed view of current AmlCompute status, use get_status()
    print(compute_target.get_status().serialize())



In [17]:
compute_target=ws.compute_targets['FirstVM']
if compute_target and type(compute_target) is AmlCompute:
    print("found compute target: " + compute_name)
else:
    print("creating new compute target...")
    provisioning_config = AmlCompute.provisioning_configuration(vm_size = vm_size,min_nodes = 0, max_nodes =4)
    # if no min node count is provided it will use the scale settings for the cluster
    compute_target.wait_for_completion(show_output=True)
    print(compute_target.get_status().serialize())

    

creating new compute target...

Running
{'errors': [], 'creationTime': '2020-11-06T04:47:37.338916+00:00', 'createdBy': {'userObjectId': 'aa4eabb9-da68-4dec-bff2-3897e808ce7a', 'userTenantId': '660b3398-b80e-49d2-bc5b-ac1dc93b5254', 'userName': None}, 'modifiedTime': '2020-11-06T04:51:55.253989+00:00', 'state': 'Running', 'vmSize': 'STANDARD_D3_V2'}


In [21]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform,normal,choice
import os

# Specify parameter sampler
ps = RandomParameterSampling( {
        "learning_rate": normal(10, 3),
        "keep_probability": uniform(0.05, 0.1),
        "batch_size": choice(16, 32, 64, 128)
    }
)


# Specify a Policy

policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1) ### YOUR CODE HERE ###

if "training" not in os.listdir():
    os.mkdir("./training")

    
"""    



"""

"    \n# Create a SKLearn estimator for use with train.py\nest = ### YOUR CODE HERE ###\nimport shutil\nshutil.copy('./train.py',source_directory)\n\nest = Estimator(source_directory=source_directory, \n                compute_target=compute_target, \n                entry_script='train.py', \n                conda_packages=['scikit-learn'],\n               )\n\n# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.\n### YOUR CODE HERE ###\n\n\nhyperdrive_config = HyperDriveConfig(estimator=est,\n                                hyperparameter_sampling=ps,\n                                policy=policy,\n                                primary_metric_name='validation_acc',\n                                primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,\n                                max_total_runs=4,\n                                max_concurrent_runs=4)\n"

In [23]:
source_directory=os.getcwd()+"/training"

In [87]:
# Create a SKLearn estimator for use with train.py
from azureml.train.estimator import Estimator
import shutil
shutil.copy('./train.py',source_directory)

est = Estimator(source_directory=source_directory, 
                compute_target='local', 
                entry_script='train.py', 
                conda_packages=['scikit-learn'],
               )

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
### YOUR CODE HERE ###


In [88]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
exp_hyper=Experiment(ws,"hyper_test")
model=exp_hyper.submit(hyperdrive_config,show_output=True)
model.wait_for_completion(show_output=True)


RunId: HD_2a6c1928-191b-4b7e-8d39-857cccdabecc
Web View: https://ml.azure.com/experiments/hyper_test/runs/HD_2a6c1928-191b-4b7e-8d39-857cccdabecc?wsid=/subscriptions/82648f26-b738-43a4-9ebb-f954c9f1ff3a/resourcegroups/aml-quickstarts-124826/workspaces/quick-starts-ws-124826

Streaming azureml-logs/hyperdrive.txt

"<START>[2020-11-06T07:03:34.796430][API][INFO]Experiment created<END>\n""<START>[2020-11-06T07:03:35.537012][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2020-11-06T07:03:35.889277][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"<START>[2020-11-06T07:03:36.5587864Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>

Execution Summary
RunId: HD_2a6c1928-191b-4b7e-8d39-857cccdabecc
Web View: https://ml.azure.com/experiments/hyper_test/runs/HD_2a6c1928-191b-4b7e-8d39-857cccdabecc?wsid=/subscriptions/82648f26-b738-4

ActivityFailedException: ActivityFailedException:
	Message: Activity Failed:
{
    "error": {
        "code": "UserError",
        "message": "User errors were found in at least one of the child runs.",
        "messageParameters": {},
        "details": []
    },
    "time": "0001-01-01T00:00:00.000Z"
}
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Activity Failed:\n{\n    \"error\": {\n        \"code\": \"UserError\",\n        \"message\": \"User errors were found in at least one of the child runs.\",\n        \"messageParameters\": {},\n        \"details\": []\n    },\n    \"time\": \"0001-01-01T00:00:00.000Z\"\n}"
    }
}

In [69]:
model

Experiment,Id,Type,Status,Details Page,Docs Page
hyper_test,HD_c3b640f8-ec78-490e-be10-f0931f62de85,hyperdrive,Running,Link to Azure Machine Learning studio,Link to Documentation


In [75]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
filename = 'model.joblib'
joblib.dump(exp_hyper, filename)

TypeError: can't pickle _thread.RLock objects

In [42]:
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core import Dataset, Datastore

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
df=Dataset.Tabular.from_delimited_files(path="https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv")

In [44]:
df.to_pandas_dataframe()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,57,technician,married,high.school,no,no,yes,cellular,may,mon,...,1,999,1,failure,-1.8,92.893,-46.2,1.299,5099.1,no
1,55,unknown,married,unknown,unknown,yes,no,telephone,may,thu,...,2,999,0,nonexistent,1.1,93.994,-36.4,4.860,5191.0,no
2,33,blue-collar,married,basic.9y,no,no,no,cellular,may,fri,...,1,999,1,failure,-1.8,92.893,-46.2,1.313,5099.1,no
3,36,admin.,married,high.school,no,no,no,telephone,jun,fri,...,4,999,0,nonexistent,1.4,94.465,-41.8,4.967,5228.1,no
4,27,housemaid,married,high.school,no,yes,no,cellular,jul,fri,...,2,999,0,nonexistent,1.4,93.918,-42.7,4.963,5228.1,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32945,56,housemaid,married,basic.4y,no,no,yes,cellular,jul,mon,...,1,999,0,nonexistent,1.4,93.918,-42.7,4.960,5228.1,no
32946,37,management,married,university.degree,no,no,yes,cellular,jul,fri,...,7,999,0,nonexistent,1.4,93.918,-42.7,4.957,5228.1,no
32947,26,admin.,single,university.degree,no,no,no,cellular,may,tue,...,4,999,1,failure,-1.8,92.893,-46.2,1.266,5099.1,no
32948,31,blue-collar,single,basic.9y,no,no,no,cellular,apr,mon,...,1,999,0,nonexistent,-1.8,93.075,-47.1,1.405,5099.1,no


In [48]:
#from train import clean_data
import pandas as pd
# Use the clean_data function to clean your data.
def clean_data(data):
    # Dict for cleaning data
    months = {"jan":1, "feb":2, "mar":3, "apr":4, "may":5, "jun":6, "jul":7, "aug":8, "sep":9, "oct":10, "nov":11, "dec":12}
    weekdays = {"mon":1, "tue":2, "wed":3, "thu":4, "fri":5, "sat":6, "sun":7}

    # Clean and one hot encode data
    x_df = data.to_pandas_dataframe().dropna()
    jobs = pd.get_dummies(x_df.job, prefix="job")
    x_df.drop("job", inplace=True, axis=1)
    x_df = x_df.join(jobs)
    x_df["marital"] = x_df.marital.apply(lambda s: 1 if s == "married" else 0)
    x_df["default"] = x_df.default.apply(lambda s: 1 if s == "yes" else 0)
    x_df["housing"] = x_df.housing.apply(lambda s: 1 if s == "yes" else 0)
    x_df["loan"] = x_df.loan.apply(lambda s: 1 if s == "yes" else 0)
    contact = pd.get_dummies(x_df.contact, prefix="contact")
    x_df.drop("contact", inplace=True, axis=1)
    x_df = x_df.join(contact)
    education = pd.get_dummies(x_df.education, prefix="education")
    x_df.drop("education", inplace=True, axis=1)
    x_df = x_df.join(education)
    x_df["month"] = x_df.month.map(months)
    x_df["day_of_week"] = x_df.day_of_week.map(weekdays)
    x_df["poutcome"] = x_df.poutcome.apply(lambda s: 1 if s == "success" else 0)

    y_df = x_df.pop("y").apply(lambda s: 1 if s == "yes" else 0)
    return x_df,y_df
x, y = clean_data(df)

0        0
1        0
2        0
3        0
4        0
        ..
32945    0
32946    0
32947    0
32948    0
32949    0
Name: y, Length: 32950, dtype: int64

In [68]:
from azureml.train.automl import AutoMLConfig
import numpy as np


# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric="AUC_weighted",
    training_data=x,
    label_column_name="y",
    n_cross_validations=3)


In [67]:
# Submit your automl run
exp=Experiment(ws,"automl_test")
automl_model=exp.submit(automl_config,show_output=True)
### YOUR CODE HERE ###
import joblib



ValidationException: ValidationException:
	Message: Install the required versions of packages using the requirements file. Requirements file location: /anaconda/envs/azureml_py36/lib/python3.6/site-packages/azureml/automl/core/validated_linux_requirements.txt. Alternatively, use remote target to avoid dependency management. 
Required version/Installed version
cryptography<=3.1.1/cryptography 3.2
zipp<=3.3.1/zipp 3.4.0
	InnerException: None
	ErrorResponse 
{
    "error": {
        "code": "UserError",
        "message": "Install the required versions of packages using the requirements file. Requirements file location: /anaconda/envs/azureml_py36/lib/python3.6/site-packages/azureml/automl/core/validated_linux_requirements.txt. Alternatively, use remote target to avoid dependency management. \nRequired version/Installed version\ncryptography<=3.1.1/cryptography 3.2\nzipp<=3.3.1/zipp 3.4.0",
        "inner_error": {
            "code": "NotSupported",
            "inner_error": {
         

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###

joblib.dump(automl_model, 'autiml_model1.sav')